![logog](https://raw.githubusercontent.com/Pacific-AI-Corp/langtest/main/docs/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pacific-AI-Corp/langtest/blob/main/demo/tutorials/llm_notebooks/dataset-notebooks/CommonsenseQA_dataset.ipynb)

**LangTest** is an open-source python library designed to help developers deliver safe and effective Natural Language Processing (NLP) models. Whether you are using **John Snow Labs, Hugging Face, Spacy** models or **OpenAI, Cohere, AI21, Hugging Face Inference API and Azure-OpenAI** based LLMs, it has got you covered. You can test any Named Entity Recognition (NER), Text Classification, fill-mask, Translation model using the library. We also support testing LLMS for Question-Answering, Summarization and text-generation tasks on benchmark datasets. The library supports 60+ out of the box tests. For a complete list of supported test categories, please refer to the [documentation](http://langtest.org/docs/pages/docs/test_categories).

Metrics are calculated by comparing the model's extractions in the original list of sentences against the extractions carried out in the noisy list of sentences. The original annotated labels are not used at any point, we are simply comparing the model against itself in a 2 settings.

# Getting started with LangTest

In [1]:
!pip install "langtest[openai,transformers,evaluate]"

# Harness and Its Parameters

The Harness class is a testing class for Natural Language Processing (NLP) models. It evaluates the performance of a NLP model on a given task using test data and generates a report with test results.Harness can be imported from the LangTest library in the following way.

In [2]:
#Import Harness from the LangTest library
from langtest import Harness

It imports the Harness class from within the module, that is designed to provide a blueprint or framework for conducting NLP testing, and that instances of the Harness class can be customized or configured for different testing scenarios or environments.

Here is a list of the different parameters that can be passed to the Harness function:

<br/>


| Parameter  | Description |  
| - | - |
|**task**     |Task for which the model is to be evaluated (question-answering or summarization)|
| **model**     | Specifies the model(s) to be evaluated. This parameter can be provided as either a dictionary or a list of dictionaries. Each dictionary should contain the following keys: <ul><li>model (mandatory): 	PipelineModel or path to a saved model or pretrained pipeline/model from hub.</li><li>hub (mandatory): Hub (library) to use in back-end for loading model from public models hub or from path</li></ul>|
| **data**      | The data to be used for evaluation. A dictionary providing flexibility and options for data sources. It should include the following keys: <ul><li>data_source (mandatory): The source of the data.</li><li>subset (optional): The subset of the data.</li><li>feature_column (optional): The column containing the features.</li><li>target_column (optional): The column containing the target labels.</li><li>split (optional): The data split to be used.</li><li>source (optional): Set to 'huggingface' when loading Hugging Face dataset.</li></ul> |
| **config**    | Configuration for the tests to be performed, specified in the form of a YAML file. |

<br/>
<br/>

# OpenAI Model Testing For Question Answering

In this section, we dive into testing of OpenAI models in Question Answering task.

LangTest supports robustness tests for LLM testing for now.

In [3]:
import os

os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"

## CommonsenseQA
[CommonsenseQA: A Question Answering Challenge Targeting Commonsense Knowledge](https://arxiv.org/abs/1811.00937)

**Dataset Summary**

CommonsenseQA is a multiple-choice question answering dataset that requires different types of commonsense knowledge to predict the correct answers .

**Data Splits**

- `test` : Testing set from the CommonsenseQA dataset, containing 1140 questions. This dataset does not contain labels and accuracy & fairness tests cannot be run with it.
- `test-tiny` : Truncated version of CommonsenseQA-test dataset which contains 50 questions. This dataset does not contain labels and accuracy & fairness tests cannot be run with it.
- `validation` : validation set from the CommonsenseQA dataset, containing 1221 question and answer examples.
- `validation-tiny` : Truncated version of CommonsenseQA-validation dataset which contains 50 question and answer examples.

### Setup and Configure Harness

In [4]:
harness = Harness(
                  task="question-answering", 
                  model={"model": "gpt-3.5-turbo-instruct","hub":"openai"}, 
                  data={"data_source" :"CommonsenseQA",
                        "split":"validation-tiny"}
                  )

Test Configuration : 
 {
 "model_parameters": {
  "max_tokens": 64
 },
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "lowercase": {
    "min_pass_rate": 0.7
   }
  }
 }
}


## Robustness

For tests we used uppercase, Dyslexia Word Swap, Add Slangs, Insert Abbreviations and Speech to Text typos . Other available robustness tests for QA task are:
* `add_context`
* `add_contraction`
* `add_punctuation`
* `add_typo`
* `add_ocr_typo`
* `american_to_british`
* `british_to_american`
* `lowercase`
* `strip_punctuation`
* `titlecase`
* `uppercase`
* `number_to_word`
* `add_abbreviation`
* `add_speech_to_text_typo`
* `add_slangs`
* `dyslexia_word_swap`
* `multiple_perturbations`
* `adjective_synonym_swap`
* `adjective_antonym_swap`
* `strip_all_punctuation`

You can also set prompts and other model parameters in config. Possible parameters are:
* `user_promt:` Promt to be given to the model.
* `temperature:` Temperature of the model.
* `max_tokens:` Maximum number of output tokens allowed for model.

In [5]:
harness.configure(
{
 'tests': {'defaults': {'min_pass_rate': 0.65},
           'robustness': {'uppercase': {'min_pass_rate': 0.66},
                          'dyslexia_word_swap':{'min_pass_rate': 0.60}

                        }
          }
 }
 )

{'tests': {'defaults': {'min_pass_rate': 0.65},
  'robustness': {'uppercase': {'min_pass_rate': 0.66},
   'dyslexia_word_swap': {'min_pass_rate': 0.6}}}}

➤ You can adjust the level of transformation in the sentence by using the "`prob`" parameter, which controls the proportion of words to be changed during robustness tests.

➤ **NOTE** : "`prob`" defaults to 1.0, which means all words will be transformed.
```
harness.configure(
{
 'tests': {
    'defaults': {'min_pass_rate': 0.65},
      'robustness': {
        'uppercase': {'min_pass_rate': 0.66, 'prob': 0.50},
        'dyslexia_word_swap':{'min_pass_rate': 0.60, 'prob': 0.70},
      }
  }
})

```

Here we have configured the harness to perform Five robustness tests and defined the minimum pass rate for each test.

In [6]:
harness.data = harness.data[:20]

### Generating the test cases.

In [7]:
harness.generate()

Generating testcases...: 100%|██████████| 1/1 [00:00<00:00, 1000.31it/s]
[W010] - Test 'dyslexia_word_swap': 7 samples removed out of 20



In [9]:
harness.testcases()

,category,test_type,original_question,perturbed_question,options
0,robustness,uppercase,A revolving door is convenient for two directi...,A REVOLVING DOOR IS CONVENIENT FOR TWO DIRECTI...,\nA. bank\nB. library\nC. department store\nD....
1,robustness,uppercase,What do people aim to do at work?,WHAT DO PEOPLE AIM TO DO AT WORK?,\nA. complete job\nB. learn from each other\nC...
2,robustness,uppercase,Where would you find magazines along side many...,WHERE WOULD YOU FIND MAGAZINES ALONG SIDE MANY...,\nA. doctor\nB. bookstore\nC. market\nD. train...
3,robustness,uppercase,Where are you likely to find a hamburger?,WHERE ARE YOU LIKELY TO FIND A HAMBURGER?,\nA. fast food restaurant\nB. pizza\nC. ground...
4,robustness,uppercase,James was looking for a good place to buy farm...,JAMES WAS LOOKING FOR A GOOD PLACE TO BUY FARM...,\nA. midwest\nB. countryside\nC. estate\nD. fa...
5,robustness,uppercase,What island country is ferret popular?,WHAT ISLAND COUNTRY IS FERRET POPULAR?,\nA. own home\nB. north carolina\nC. great bri...
6,robustness,uppercase,In what Spanish speaking North American countr...,IN WHAT SPANISH SPEAKING NORTH AMERICAN COUNTR...,\nA. mildred's coffee shop\nB. mexico\nC. dine...
7,robustness,uppercase,What do animals do when an enemy is approaching?,WHAT DO ANIMALS DO WHEN AN ENEMY IS APPROACHING?,\nA. feel pleasure\nB. procreate\nC. pass wate...
8,robustness,uppercase,Reading newspaper one of many ways to practice...,READING NEWSPAPER ONE OF MANY WAYS TO PRACTICE...,\nA. literacy\nB. knowing how to read\nC. mone...
9,robustness,uppercase,What do people typically do while playing guitar?,WHAT DO PEOPLE TYPICALLY DO WHILE PLAYING GUITAR?,\nA. cry\nB. hear sounds\nC. singing\nD. arthr...


harness.generate() method automatically generates the test cases (based on the provided configuration)

### Running the tests

In [10]:
harness.run()

Running testcases... : 100%|██████████| 33/33 [00:33<00:00,  1.03s/it]


Called after harness.generate() and is to used to run all the tests.  Returns a pass/fail flag for each test.

### Generated Results

In [11]:
harness.generated_results()

,category,test_type,original_question,perturbed_question,options,expected_result,actual_result,pass
0,robustness,uppercase,A revolving door is convenient for two directi...,A REVOLVING DOOR IS CONVENIENT FOR TWO DIRECTI...,\nA. bank\nB. library\nC. department store\nD....,A. bank,A. bank,True
1,robustness,uppercase,What do people aim to do at work?,WHAT DO PEOPLE AIM TO DO AT WORK?,\nA. complete job\nB. learn from each other\nC...,A. complete job,A. complete job,True
2,robustness,uppercase,Where would you find magazines along side many...,WHERE WOULD YOU FIND MAGAZINES ALONG SIDE MANY...,\nA. doctor\nB. bookstore\nC. market\nD. train...,B. Bookstore,B. bookstore,True
3,robustness,uppercase,Where are you likely to find a hamburger?,WHERE ARE YOU LIKELY TO FIND A HAMBURGER?,\nA. fast food restaurant\nB. pizza\nC. ground...,A. fast food restaurant,A. fast food restaurant,True
4,robustness,uppercase,James was looking for a good place to buy farm...,JAMES WAS LOOKING FOR A GOOD PLACE TO BUY FARM...,\nA. midwest\nB. countryside\nC. estate\nD. fa...,D. farming areas,D. farming areas,True
5,robustness,uppercase,What island country is ferret popular?,WHAT ISLAND COUNTRY IS FERRET POPULAR?,\nA. own home\nB. north carolina\nC. great bri...,C. Great Britain,C. Great Britain,True
6,robustness,uppercase,In what Spanish speaking North American countr...,IN WHAT SPANISH SPEAKING NORTH AMERICAN COUNTR...,\nA. mildred's coffee shop\nB. mexico\nC. dine...,B. Mexico,B. Mexico,True
7,robustness,uppercase,What do animals do when an enemy is approaching?,WHAT DO ANIMALS DO WHEN AN ENEMY IS APPROACHING?,\nA. feel pleasure\nB. procreate\nC. pass wate...,D. listen to each other,D. listen to each other,True
8,robustness,uppercase,Reading newspaper one of many ways to practice...,READING NEWSPAPER ONE OF MANY WAYS TO PRACTICE...,\nA. literacy\nB. knowing how to read\nC. mone...,A. literacy,A. literacy,True
9,robustness,uppercase,What do people typically do while playing guitar?,WHAT DO PEOPLE TYPICALLY DO WHILE PLAYING GUITAR?,\nA. cry\nB. hear sounds\nC. singing\nD. arthr...,E. Making music,E. making music,True


This method returns the generated results in the form of a pandas dataframe, which provides a convenient and easy-to-use format for working with the test results. You can use this method to quickly identify the test cases that failed and to determine where fixes are needed.

### Final Results

We can call `.report()` which summarizes the results giving information about pass and fail counts and overall test pass/fail flag.

In [12]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,robustness,uppercase,1,19,95%,66%,True
1,robustness,dyslexia_word_swap,2,11,85%,60%,True


## Fairness

Available Fairness tests for QA task are:

* `max_gender_rouge1_score`
* `max_gender_rouge2_score`
* `max_gender_rougeL_score`
* `max_gender_rougeLsum_score`
* `min_gender_rouge1_score`
* `min_gender_rouge2_score`
* `min_gender_rougeL_score`
* `min_gender_rougeLsum_score`

In [13]:
harness = Harness(
                  task="question-answering", 
                  model={"model": "gpt-3.5-turbo-instruct","hub":"openai"}, 
                  data={"data_source" :"CommonsenseQA",
                        "split":"validation-tiny"}
                  )

Test Configuration : 
 {
 "model_parameters": {
  "max_tokens": 64
 },
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "lowercase": {
    "min_pass_rate": 0.7
   }
  }
 }
}


In [14]:
harness.configure(
{
 'tests': {'defaults': {'min_pass_rate': 0.65},
           'fairness': {
               'min_gender_rouge1_score': {'min_score': 0.66},
               'min_gender_rouge2_score':{'min_score': 0.60},
               'min_gender_rougeL_score': {'min_score': 0.66},
               'min_gender_rougeLsum_score': {'min_score': 0.66},
               'max_gender_rouge1_score': {'max_score': 0.66},
               'max_gender_rouge2_score':{'max_score': 0.60},
               'max_gender_rougeL_score': {'max_score': 0.66},
               'max_gender_rougeLsum_score': {'max_score': 0.66},




                        }
          }
 }
 )

{'tests': {'defaults': {'min_pass_rate': 0.65},
  'fairness': {'min_gender_rouge1_score': {'min_score': 0.66},
   'min_gender_rouge2_score': {'min_score': 0.6},
   'min_gender_rougeL_score': {'min_score': 0.66},
   'min_gender_rougeLsum_score': {'min_score': 0.66},
   'max_gender_rouge1_score': {'max_score': 0.66},
   'max_gender_rouge2_score': {'max_score': 0.6},
   'max_gender_rougeL_score': {'max_score': 0.66},
   'max_gender_rougeLsum_score': {'max_score': 0.66}}}}

In [15]:
harness.data=harness.data[:30]

### Generating the Test Cases

In [16]:
harness.generate()

Generating testcases...: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [18]:
harness.testcases()

,category,test_type,test_case
0,fairness,min_gender_rouge1_score,male
1,fairness,min_gender_rouge1_score,female
2,fairness,min_gender_rouge1_score,unknown
3,fairness,min_gender_rouge2_score,male
4,fairness,min_gender_rouge2_score,female
5,fairness,min_gender_rouge2_score,unknown
6,fairness,min_gender_rougeL_score,male
7,fairness,min_gender_rougeL_score,female
8,fairness,min_gender_rougeL_score,unknown
9,fairness,min_gender_rougeLsum_score,male


### Running the tests

In [19]:
harness.run()

Running testcases... : 100%|██████████| 24/24 [01:18<00:00,  2.72s/it]

### Generated Results

In [20]:
harness.generated_results()

,category,test_type,test_case,expected_result,actual_result,pass
0,fairness,min_gender_rouge1_score,male,0.66,0.800000,True
1,fairness,min_gender_rouge1_score,female,0.66,1.000000,True
2,fairness,min_gender_rouge1_score,unknown,0.66,0.791667,True
3,fairness,min_gender_rouge2_score,male,0.60,0.800000,True
4,fairness,min_gender_rouge2_score,female,0.60,1.000000,True
5,fairness,min_gender_rouge2_score,unknown,0.60,0.791667,True
6,fairness,min_gender_rougeL_score,male,0.66,0.800000,True
7,fairness,min_gender_rougeL_score,female,0.66,1.000000,True
8,fairness,min_gender_rougeL_score,unknown,0.66,0.791667,True
9,fairness,min_gender_rougeLsum_score,male,0.66,0.800000,True


### Final Results

In [21]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,fairness,min_gender_rouge1_score,0,3,100%,65%,True
1,fairness,min_gender_rouge2_score,0,3,100%,65%,True
2,fairness,min_gender_rougeL_score,0,3,100%,65%,True
3,fairness,min_gender_rougeLsum_score,0,3,100%,65%,True
4,fairness,max_gender_rouge1_score,3,0,0%,65%,False
5,fairness,max_gender_rouge2_score,3,0,0%,65%,False
6,fairness,max_gender_rougeL_score,3,0,0%,65%,False
7,fairness,max_gender_rougeLsum_score,3,0,0%,65%,False


## Accuracy

Available Accuracy tests for QA task are:

* `min_exact_match_score`
* `min_bleu_score`
* `min_rouge1_score`
* `min_rouge2_score`
* `min_rougeL_score`
* `min_rougeLsum_score`

In [22]:
harness = Harness(
                  task="question-answering", 
                  model={"model": "gpt-3.5-turbo-instruct","hub":"openai"}, 
                  data={"data_source" :"CommonsenseQA",
                        "split":"validation-tiny"}
                  )

Test Configuration : 
 {
 "model_parameters": {
  "max_tokens": 64
 },
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "lowercase": {
    "min_pass_rate": 0.7
   }
  }
 }
}


In [23]:
harness.configure(
{
 'tests': {'defaults': {'min_pass_rate': 0.65},
          'accuracy': {'min_exact_match_score': {'min_score': 0.80},
                        'min_rouge1_score':{'min_score': 0.80},
                        'min_rougeL_score':{'min_score': 0.80},
                        'min_bleu_score':{'min_score': 0.80},
                        'min_rouge2_score':{'min_score': 0.80},
                        'min_rougeLsum_score':{'min_score': 0.80}

                        }
          }
 }
 )

{'tests': {'defaults': {'min_pass_rate': 0.65},
  'accuracy': {'min_exact_match_score': {'min_score': 0.8},
   'min_rouge1_score': {'min_score': 0.8},
   'min_rougeL_score': {'min_score': 0.8},
   'min_bleu_score': {'min_score': 0.8},
   'min_rouge2_score': {'min_score': 0.8},
   'min_rougeLsum_score': {'min_score': 0.8}}}}

In [24]:
harness.data=harness.data[:30]

### Generating the test cases.

In [25]:
harness.generate()

Generating testcases...: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [26]:
harness.testcases()

,category,test_type
0,accuracy,min_exact_match_score
1,accuracy,min_rouge1_score
2,accuracy,min_rougeL_score
3,accuracy,min_bleu_score
4,accuracy,min_rouge2_score
5,accuracy,min_rougeLsum_score


### Running the tests

In [27]:
harness.run()

Running testcases... : 100%|██████████| 6/6 [00:30<00:00,  5.11s/it]


### Generated Results

In [28]:
harness.generated_results()

,category,test_type,expected_result,actual_result,pass
0,accuracy,min_exact_match_score,0.8,0.566667,False
1,accuracy,min_rouge1_score,0.8,0.833333,True
2,accuracy,min_rougeL_score,0.8,0.833333,True
3,accuracy,min_bleu_score,0.8,0.645353,False
4,accuracy,min_rouge2_score,0.8,0.816667,True
5,accuracy,min_rougeLsum_score,0.8,0.822222,True


### Final Results

In [29]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,accuracy,min_exact_match_score,1,0,0%,65%,False
1,accuracy,min_rouge1_score,0,1,100%,65%,True
2,accuracy,min_rougeL_score,0,1,100%,65%,True
3,accuracy,min_bleu_score,1,0,0%,65%,False
4,accuracy,min_rouge2_score,0,1,100%,65%,True
5,accuracy,min_rougeLsum_score,0,1,100%,65%,True
